In [17]:
import numpy as np
file_names=[]
all_shap=[]
for i in range(140):
    file_name="/home/zyxing/adult_income/shaps/clf"+str(i)+".ptl.npy"
    
    try:
        all_shap.append(np.load(file_name))
        file_names.append(file_name)
    except Exception as e:
        continue
leng=len(all_shap)
all_shap=np.array(all_shap)


In [18]:
all_shap.shape

(94, 22792, 14)

In [19]:
import numpy as np
#定义L1损失函数
def L1_loss(y_true,y_pre): 
    return np.sum(np.abs(y_true-y_pre))
#定义L2损失函数
def L2_loss(y_true,y_pre):
    return np.sum(np.square(y_true-y_pre))

In [20]:
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression

# X_train, X_test, y_train, y_test = train_test_split(all_input_shap, shap_mean, test_size = 0.3, random_state = 0)
# model= LinearRegression()

# model.fit(X_train,y_train)

# model.singular_=np.array([0.2]*5)
# from sklearn.metrics import accuracy_score
# y_pred=model.predict(X_test)
#l2 148
# L2_loss(y_pred,y_test)
# np.sum(abs(y_pred))


In [21]:
from torch.utils.data import Dataset,DataLoader
import torch
import torch.nn as nn
class MyDataSet(Dataset):
    def __init__(self,data_x,data_y):
        self.x=torch.tensor(data_x).to(torch.float)
        self.y=torch.tensor(data_y).to(torch.float)
        # print(self.x.shape,self.y.shape)
    def __getitem__(self, item):
        return self.x[item],self.y[item],item
    def __len__(self):
        return len(self.x)
class MyNetWork(nn.Module):
    def __init__(self):
        super(MyNetWork,self).__init__()
        self.fc1=nn.Linear(5,1)
        # self.fc2=nn.Linear(2,1)
    def forward(self,x):
        x=self.fc1(x)
        # x=self.fc2(x)
        return x

In [22]:
import pandas as pd
from numpy import int64

df=pd.read_csv("adult.csv",encoding='latin-1')
df[df == '?'] = np.nan
labels = df['income']
labels=labels[:int(labels.shape[0]*0.7)].copy()
labels[labels=="<=50K"]=0
labels[labels==">50K"]=1
label=labels.astype(int64)
label=np.array(label)

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
y_shaps=[]
x_shaps=[]
out_shaps=[]
classes=[]
def getLine(seed:int):
    rand_ints=np.random.choice(leng,5,replace=False)
    print(file_names[rand_ints])
    all_input_shap=all_shap[rand_ints,:,:]
    all_input_shap=all_input_shap.transpose([1,2,0])
    shap_mean=np.mean(all_shap,axis=0)
    # print(all_input_shap.shape,shap_mean.shape)
    # print(all_shap.shape,all_input_shap.shape,shap_mean.shape)
    if seed==-1:
        train_X, test_X, train_Y, test_Y = train_test_split(all_input_shap, shap_mean, test_size = 0.3, shuffle=False)
    else:
        train_X, test_X, train_Y, test_Y = train_test_split(all_input_shap, shap_mean, test_size = 0.3, random_state=seed)
    train_DataSet=MyDataSet(train_X,train_Y)
    test_DataSet=MyDataSet(test_X,test_Y)
    batch_size=3
    print(train_X.shape)
    train_dataloader = DataLoader(train_DataSet,batch_size=batch_size,shuffle=True)
    test_dataloader = DataLoader(test_DataSet,batch_size=batch_size,shuffle=True)
    net=MyNetWork()
    sta_dic=net.state_dict()
    sta_dic['fc1.weight']=torch.tensor([[0.2, 0.2, 0.2, 0.2, 0.2]])
    sta_dic['fc1.bias']=torch.tensor([0])

    net.load_state_dict(sta_dic)
    import time
    import torch.optim as optim
    criterion=nn.L1Loss()
    optimizer = optim.Adam(net.parameters(), lr=0.0001)
    start = time.time()
    for epoch in range(10):

        running_loss = 0.0
        for i, data in enumerate(train_dataloader, 0):
            # 获取输入数据
            inputs, label,_= data
            # 清空梯度缓存
            optimizer.zero_grad()
            # print(inputs.shape)
            # print(torch.dist(inputs,torch.tensor(partial_shaps[index]),1))
            # inputs=inputs.transpose(1,2)
            # print(inputs.shape)
            outputs = net(inputs).squeeze()
            label=label.squeeze()
            # print(outputs.shape,label.shape)
            loss = criterion(outputs, label)
            loss.backward()
            optimizer.step()

            # 打印统计信息
            running_loss += loss.item()
            # print(loss.item())
    print('Finished Training! Total cost time: ', time.time()-start)
    all_real_input_mean_shap=np.mean(all_input_shap,axis=2)
    loss1=0
    loss2=0
    loss3=0
    torch.save(net.state_dict(),"./openXAI/net.pt")
    # print(net.state_dict())
    for i,data in enumerate(test_dataloader,0):
        x,y,index=data
        index+=15954
        # index+=train_size
        # print(index)
        # if len(index)==1:
        #     classes.append(label[index.item()])
        # else:
        #     classes.append(label[index.items()])
        # print([list(index.detach().numpy())])
        classes.append(labels[list(index.detach().numpy())])
        y=y.squeeze()
        # x=x.transpose(1,2)
        # print(torch.mean(x,dim=2).shape)
        output_tensor=net.forward(x).squeeze()
        all_real_input_mean_shap_np=np.array(torch.mean(x,dim=-1))
        #5个模型求取平均代表的就是0.2
        # print(output_tensor.shape,torch.tensor(all_real_mean_shap[index]).shape,torch.tensor(all_real_input_mean_shap[index]).shape)
        # print(x.shape)
        x_shaps.append(all_real_input_mean_shap_np)
        y_shaps.append(y.detach().numpy())
        out_shaps.append(output_tensor.detach().numpy())
        # out_shaps.append(output_tensor.detach().numpy())
        # y_shaps.append(y.detach().numpy())
        loss1+=torch.dist(output_tensor,y,2)
        loss2+=torch.dist(torch.tensor(all_real_input_mean_shap_np),y,2)
        
        # output_tensor_np=output_tensor.detach().numpy()
        # y_np=y.detach().numpy()
        
        
        # print(all_real_input_mean_shap_np.shape)
        # if len(index)==1:
        #     output_tensor_np=output_tensor_np[np.newaxis,:]
        #     y_np=y_np[np.newaxis,:]
            # all_real_input_mean_shap_np=all_real_input_mean_shap_np[np.newaxis,:]
        # for i in range(len(index)):
            
        #     loss1+=getDIff(50,output_tensor_np[i],y_np[i])
        #     loss2+=getDIff(50,all_real_input_mean_shap_np[i],y_np[i])
    ans=str(loss1.item())+"\t"+str(loss2.item())
    print(ans)
    
    # model= LinearRegression()

    # model.fit(train_X,train_Y)

    # model.singular_=np.array([0.2]*5)
    # pred_Y=model.predict(test_X)
    # print(L2_loss(pred_Y,test_Y))

    with open("same_deep","a+") as f:
        f.write(ans+"\n")


In [24]:
# for i in range(100):
#     getLine(i)
getLine(-1)

TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
x_shaps=np.concatenate(x_shaps,axis=0)

In [ ]:

y_shaps[-1]=y_shaps[-1][np.newaxis,:]
out_shaps[-1]=out_shaps[-1][np.newaxis,:]

In [ ]:
y_shaps=np.concatenate(y_shaps,axis=0)
out_shaps=np.concatenate(out_shaps,axis=0)

In [ ]:
classes=np.concatenate(classes,axis=0)

In [ ]:
print(x_shaps.shape,y_shaps.shape,out_shaps.shape,classes.shape)

(6838, 14) (6838, 14) (6838, 14) (6838,)


In [ ]:
def getDis(data):
    class0_shap=[]
    class1_shap=[]
    for label,data in zip(classes,data):
        if label==0:
            class0_shap.append(data)
        elif label==1:
            class1_shap.append(data)
        else:
            print(label)

    class0_shap=np.array(class0_shap)
    class1_shap=np.array(class1_shap)

    return np.mean(np.std(class0_shap,axis=0)),np.mean(np.std(class1_shap,axis=0)),np.mean(np.std(data,axis=0))

In [ ]:
import networkx as nx
def getGraph(datas,file_name):
    G=nx.Graph()
    for i,(label,data) in enumerate(zip(classes,datas)):
        G.add_node(i,label=label,data=data)
        for j in range(i):
            G.add_edge(i,j,weights=L2_loss(G.nodes[i]["data"],G.nodes[j]["data"]))
    nx.write_gpickle(G,file_name)

In [ ]:
# getGraph(x_shaps,"adult_x_shaos.gpickle")
# getGraph(y_shaps,"adult_y_shaos.gpickle")
# getGraph(out_shaps,"adult_out_shaos.gpickle")